In [79]:
tok='[CLS] yhdysvaltain sen+ +aatti on hyväksy+ +nyt pitkän kiist+ +ely+ +n velka+ +kat+ +on nosta+ +en ääni+ +n seitsemänkymmentä+ +neljä kaksikymmentä kuusi [SEP]'


In [1]:
import logging
import sys
import os
import lzma
import random


import collections
#import tokenization
import tensorflow as tf

import copy
import json
import math
import re
import numpy as np
import six

In [80]:
tokk=tok.split()

In [81]:
class TrainingInstance(object):
  """A single training instance (sentence pair)."""

  def __init__(self,tokens,masked_lm_positions, masked_lm_labels):
    self.tokens = tokens
    #self.segment_ids = segment_ids
    #self.is_random_next = is_random_next
    self.masked_lm_positions = masked_lm_positions
    self.masked_lm_labels = masked_lm_labels

  def __str__(self):
    s = ""
    s += "tokens: %s\n" % (" ".join(
        [printable_text(x) for x in self.tokens]))
    #s += "segment_ids: %s\n" % (" ".join([str(x) for x in self.segment_ids]))
    #s += "is_random_next: %s\n" % self.is_random_next
    s += "masked_lm_positions: %s\n" % (" ".join(
        [str(x) for x in self.masked_lm_positions]))
    s += "masked_lm_labels: %s\n" % (" ".join(
        [printable_text(x) for x in self.masked_lm_labels]))
    s += "\n"
    return s

  def __repr__(self):
    return self.__str__()
def printable_text(text):
  """Returns text encoded in a way suitable for print or `tf.logging`."""

  # These functions want `str` for both Python2 and Python3, but in one case
  # it's a Unicode string and in the other it's a byte string.
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text
    elif isinstance(text, unicode):
      return text.encode("utf-8")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
      raise ValueError("Not running on Python2 or Python 3?")

In [82]:
tokk

['[CLS]',
 'yhdysvaltain',
 'sen+',
 '+aatti',
 'on',
 'hyväksy+',
 '+nyt',
 'pitkän',
 'kiist+',
 '+ely+',
 '+n',
 'velka+',
 '+kat+',
 '+on',
 'nosta+',
 '+en',
 'ääni+',
 '+n',
 'seitsemänkymmentä+',
 '+neljä',
 'kaksikymmentä',
 'kuusi',
 '[SEP]']

In [83]:
cand_indices=[]
for (i, token) in enumerate(tokk):
    if token == "[CLS]" or token == "[SEP]":
            continue
    cand_indices.append(i)
covered_indices=[]
output_tokens=[]
#masked_lm_positions = []
for i in cand_indices:
    if i in covered_indices:
        continue
    covered_indices.append(i)    
    temp_sent=tokk[:]
    #print(temp_sent)
    temp_sent[i]='[MASK]'
    
    #print(temp_sent)
    masked_lm_positions=[i]
    masked_lm_labels=[tokk[i]]
    op=TrainingInstance(temp_sent,masked_lm_positions,masked_lm_labels)
    output_tokens.append(op)
    continue

In [84]:
tokk[0]

'[CLS]'

In [85]:
def create_int_feature(values):
  feature = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
  return feature

def create_float_feature(values):
  feature = tf.train.Feature(float_list=tf.train.FloatList(value=list(values)))
  return feature

def write_instance_to_example_files(instances, max_seq_length,
                                    max_predictions_per_seq, output_files,vocab):
  """Create TF example files from `TrainingInstance`s."""
  writers = []
  for output_file in output_files:
    writers.append(tf.python_io.TFRecordWriter(output_file))

  writer_index = 0

  total_written = 0
  for (inst_index, instance) in enumerate(instances):
    input_ids =convert_tokens_to_ids(instance.tokens,vocab)
    input_mask = [1] * len(input_ids)
    #segment_ids = list(instance.segment_ids)
    assert len(input_ids) <= max_seq_length

    while len(input_ids) < max_seq_length:
      input_ids.append(0)
      input_mask.append(0)
      #segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    #assert len(segment_ids) == max_seq_length

    masked_lm_positions = list(instance.masked_lm_positions)
    masked_lm_ids = convert_tokens_to_ids(instance.masked_lm_labels,vocab)
    masked_lm_weights = [1.0] * len(masked_lm_ids)

    while len(masked_lm_positions) < max_predictions_per_seq:
      masked_lm_positions.append(0)
      masked_lm_ids.append(0)
      masked_lm_weights.append(0.0)

    #next_sentence_label = 1 if instance.is_random_next else 0

    features = collections.OrderedDict()
    features["input_ids"] = create_int_feature(input_ids)
    features["input_mask"] = create_int_feature(input_mask)
    #features["segment_ids"] = create_int_feature(segment_ids)
    features["masked_lm_positions"] = create_int_feature(masked_lm_positions)
    features["masked_lm_ids"] = create_int_feature(masked_lm_ids)
    features["masked_lm_weights"] = create_float_feature(masked_lm_weights)
    #features["next_sentence_labels"] = create_int_feature([next_sentence_label])

    tf_example = tf.train.Example(features=tf.train.Features(feature=features))

    writers[writer_index].write(tf_example.SerializeToString())
    writer_index = (writer_index + 1) % len(writers)

    total_written += 1

    if inst_index < 20:
      tf.logging.info("*** Example ***")
      tf.logging.info("tokens: %s" % " ".join(
          [printable_text(x) for x in instance.tokens]))

      for feature_name in features.keys():
        feature = features[feature_name]
        values = []
        if feature.int64_list.value:
          values = feature.int64_list.value
        elif feature.float_list.value:
          values = feature.float_list.value
        tf.logging.info(
            "%s: %s" % (feature_name, " ".join([str(x) for x in values])))

  for writer in writers:
    writer.close()

  tf.logging.info("Wrote %d total instances", total_written)

def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    index = 0
    with open(vocab_file, "r", encoding="utf-8") as reader:
        while True:
            token = reader.readline()
            if not token:
                break
            token = token.strip()
            vocab[token] = index
            index += 1
    return vocab

def convert_tokens_to_ids(tokens,vocab_words):
        """Converts a sequence of tokens into ids using the vocab."""
        ids = []
        for token in tokens:
            #print(token)
            ids.append(vocab_words[token])
            
        return ids
    

In [86]:
vocab_words=load_vocab('data/vocab_with_aff')
write_instance_to_example_files(output_tokens,300,40,['yle_examplesentence.tfrecord'],vocab_words)

INFO:tensorflow:*** Example ***
INFO:tensorflow:tokens: [CLS] [MASK] sen+ +aatti on hyväksy+ +nyt pitkän kiist+ +ely+ +n velka+ +kat+ +on nosta+ +en ääni+ +n seitsemänkymmentä+ +neljä kaksikymmentä kuusi [SEP]
INFO:tensorflow:input_ids: 2377 34521 24957 145 19353 6241 18739 21670 10649 3277 17697 32518 9444 19354 18536 3337 82 17697 24745 18071 8814 12763 24973 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:masked_lm_positions: 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:masked_lm_ids: 6241 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:masked_lm_weights: 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.

INFO:tensorflow:masked_lm_positions: 9 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:masked_lm_ids: 3277 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:masked_lm_weights: 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
INFO:tensorflow:*** Example ***
INFO:tensorflow:tokens: [CLS] yhdysvaltain sen+ +aatti on hyväksy+ +nyt pitkän kiist+ +ely+ [MASK] velka+ +kat+ +on nosta+ +en ääni+ +n seitsemänkymmentä+ +neljä kaksikymmentä kuusi [SEP]
INFO:tensorflow:input_ids: 2377 33878 24957 145 19353 6241 18739 21670 10649 3277 34521 32518 9444 19354 18536 3337 82 17697 24745 18071 8814 12763 24973 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

INFO:tensorflow:*** Example ***
INFO:tensorflow:tokens: [CLS] yhdysvaltain sen+ +aatti on hyväksy+ +nyt pitkän kiist+ +ely+ +n velka+ +kat+ +on [MASK] +en ääni+ +n seitsemänkymmentä+ +neljä kaksikymmentä kuusi [SEP]
INFO:tensorflow:input_ids: 2377 33878 24957 145 19353 6241 18739 21670 10649 3277 17697 32518 9444 19354 34521 3337 82 17697 24745 18071 8814 12763 24973 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:masked_lm_positions: 18 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:masked_lm_ids: 24745 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:masked_lm_weights: 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

In [87]:
for (inst_index, instance) in enumerate(output_tokens):
    print(instance.tokens)

['[CLS]', '[MASK]', 'sen+', '+aatti', 'on', 'hyväksy+', '+nyt', 'pitkän', 'kiist+', '+ely+', '+n', 'velka+', '+kat+', '+on', 'nosta+', '+en', 'ääni+', '+n', 'seitsemänkymmentä+', '+neljä', 'kaksikymmentä', 'kuusi', '[SEP]']
['[CLS]', 'yhdysvaltain', '[MASK]', '+aatti', 'on', 'hyväksy+', '+nyt', 'pitkän', 'kiist+', '+ely+', '+n', 'velka+', '+kat+', '+on', 'nosta+', '+en', 'ääni+', '+n', 'seitsemänkymmentä+', '+neljä', 'kaksikymmentä', 'kuusi', '[SEP]']
['[CLS]', 'yhdysvaltain', 'sen+', '[MASK]', 'on', 'hyväksy+', '+nyt', 'pitkän', 'kiist+', '+ely+', '+n', 'velka+', '+kat+', '+on', 'nosta+', '+en', 'ääni+', '+n', 'seitsemänkymmentä+', '+neljä', 'kaksikymmentä', 'kuusi', '[SEP]']
['[CLS]', 'yhdysvaltain', 'sen+', '+aatti', '[MASK]', 'hyväksy+', '+nyt', 'pitkän', 'kiist+', '+ely+', '+n', 'velka+', '+kat+', '+on', 'nosta+', '+en', 'ääni+', '+n', 'seitsemänkymmentä+', '+neljä', 'kaksikymmentä', 'kuusi', '[SEP]']
['[CLS]', 'yhdysvaltain', 'sen+', '+aatti', 'on', '[MASK]', '+nyt', 'pitkän', 'k

In [88]:
allthetokens
for tokens in output_tokens:
    

[tokens: [CLS] [MASK] sen+ +aatti on hyväksy+ +nyt pitkän kiist+ +ely+ +n velka+ +kat+ +on nosta+ +en ääni+ +n seitsemänkymmentä+ +neljä kaksikymmentä kuusi [SEP]
 masked_lm_positions: 1
 masked_lm_labels: yhdysvaltain
 ,
 tokens: [CLS] yhdysvaltain [MASK] +aatti on hyväksy+ +nyt pitkän kiist+ +ely+ +n velka+ +kat+ +on nosta+ +en ääni+ +n seitsemänkymmentä+ +neljä kaksikymmentä kuusi [SEP]
 masked_lm_positions: 2
 masked_lm_labels: sen+
 ,
 tokens: [CLS] yhdysvaltain sen+ [MASK] on hyväksy+ +nyt pitkän kiist+ +ely+ +n velka+ +kat+ +on nosta+ +en ääni+ +n seitsemänkymmentä+ +neljä kaksikymmentä kuusi [SEP]
 masked_lm_positions: 3
 masked_lm_labels: +aatti
 ,
 tokens: [CLS] yhdysvaltain sen+ +aatti [MASK] hyväksy+ +nyt pitkän kiist+ +ely+ +n velka+ +kat+ +on nosta+ +en ääni+ +n seitsemänkymmentä+ +neljä kaksikymmentä kuusi [SEP]
 masked_lm_positions: 4
 masked_lm_labels: on
 ,
 tokens: [CLS] yhdysvaltain sen+ +aatti on [MASK] +nyt pitkän kiist+ +ely+ +n velka+ +kat+ +on nosta+ +en ääni+ 

In [68]:
import os
import time
import model_1
import optimization
import tensorflow as tf

flags = tf.flags

FLAGS = flags.FLAGS
flags.DEFINE_bool("use_fp16",False, "Whether to use fp32 or fp16 arithmetic on GPU.")
flags.DEFINE_bool("do_train", False, "Whether to run training.")
flags.DEFINE_bool("amp", False, "Whether to enable AMP ops.")


In [115]:

class _LogSessionRunHook(tf.train.SessionRunHook):
  def __init__(self, global_batch_size, display_every=1000, hvd_rank=-1):
    self.global_batch_size = global_batch_size
    self.display_every = display_every
    self.hvd_rank = hvd_rank
  def after_create_session(self, session, coord):
    if FLAGS.use_fp16 or FLAGS.amp:
      print('  Step samples/sec   MLM Loss  NSP Loss  Loss  Learning-rate  Loss-scaler')
    else:
      print('  Step samples/sec   MLM Loss  NSP Loss  Loss  Learning-rate')
    self.elapsed_secs = 0.
    self.count = 0
  def before_run(self, run_context):
    self.t0 = time.time()
    if FLAGS.use_fp16 or FLAGS.amp:
      return tf.train.SessionRunArgs(
          fetches=['step_update:0', 'total_loss:0',
                   'learning_rate:0','mlm_loss:0', 'loss_scale:0'])
    else:
      return tf.train.SessionRunArgs(
          fetches=['step_update:0', 'total_loss:0',
                   'learning_rate:0','mlm_loss:0'])
  def after_run(self, run_context, run_values):
    self.elapsed_secs += time.time() - self.t0
    self.count += 1
    if FLAGS.use_fp16 or FLAGS.amp:
      global_step, total_loss, lr, mlm_loss, loss_scaler = run_values.results
    else:
      global_step, total_loss, lr, mlm_loss = run_values.results
    print_step = global_step + 1 # One-based index for printing.
    if print_step == 1 or print_step % self.display_every == 0:
        dt = self.elapsed_secs / self.count
        img_per_sec = self.global_batch_size / dt
        if self.hvd_rank >= 0:
          if FLAGS.use_fp16 or FLAGS.amp:
            print('%2d :: %6i %11.1f %10.4e %6.3f     %6.4e  %6.4e' %
                  (self.hvd_rank, print_step, img_per_sec, mlm_loss, total_loss, lr, loss_scaler))
          else:
            print('%2d :: %6i %11.1f %10.4e %6.3f     %6.4e' %
                  (self.hvd_rank, print_step, img_per_sec, mlm_loss,total_loss, lr))
        else:
          if FLAGS.use_fp16 or FLAGS.amp:
            print('%6i %11.1f %10.4e %6.3f     %6.4e  %6.4e' %
                  (print_step, img_per_sec, mlm_loss, total_loss, lr, loss_scaler))
          else:
            print('%6i %11.1f %10.4e  %6.3f     %6.4e' %
                  (print_step, img_per_sec, mlm_loss, total_loss, lr))
        self.elapsed_secs = 0.
        self.count = 0


def model_fn_builder(bert_config, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings, hvd=None):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    #segment_ids = features["segment_ids"]
    masked_lm_positions = features["masked_lm_positions"]
    masked_lm_ids = features["masked_lm_ids"]
    masked_lm_weights = features["masked_lm_weights"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    model = model_1.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        #token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings,
        compute_type=tf.float16)

    
    masked_lm_example_loss = get_masked_lm_output(
         bert_config, model.get_sequence_output(), model.get_embedding_table(),
         masked_lm_positions, masked_lm_ids, masked_lm_weights)
    #print('this stuff',masked_lm_example_loss)


    #masked_lm_loss = tf.identity(masked_lm_loss, name="mlm_loss")
    #total_loss = masked_lm_loss
    #total_loss = tf.identity(total_loss, name='total_loss')

    tvars = tf.trainable_variables()

    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint and (hvd is None or hvd.rank() == 0):
      (assignment_map, initialized_variable_names
      ) = model_1.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  %d :: name = %s, shape = %s%s", 0 if hvd is None else hvd.rank(), var.name, var.shape,
                      init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu,
          hvd, FLAGS.use_fp16, FLAGS.amp)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,
          scaffold_fn=scaffold_fn)
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(masked_lm_example_loss, masked_lm_log_probs, masked_lm_ids,
                    masked_lm_weights):
        """Computes the loss and accuracy of the model."""
        masked_lm_log_probs = tf.reshape(masked_lm_log_probs,
                                         [-1, masked_lm_log_probs.shape[-1]])
        masked_lm_predictions = tf.argmax(
            masked_lm_log_probs, axis=-1, output_type=tf.int32)
        masked_lm_example_loss = tf.reshape(masked_lm_example_loss, [-1])
        masked_lm_ids = tf.reshape(masked_lm_ids, [-1])
        masked_lm_weights = tf.reshape(masked_lm_weights, [-1])
        masked_lm_accuracy = tf.metrics.accuracy(
            labels=masked_lm_ids,
            predictions=masked_lm_predictions,
            weights=masked_lm_weights)
        masked_lm_mean_loss = tf.metrics.mean(
            values=masked_lm_example_loss, weights=masked_lm_weights)
        #log_probas=masked_lm_log_probs
        print(masked_lm_log_probs)

      

        return {
            "masked_lm_accuracy": masked_lm_accuracy,
            "masked_lm_loss": masked_lm_mean_loss
            
        }

      eval_metrics = (metric_fn, [
          masked_lm_example_loss, masked_lm_log_probs, masked_lm_ids,
          masked_lm_weights
      ])
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics,
          scaffold_fn=scaffold_fn)
    elif mode == tf.estimator.ModeKeys.PREDICT:
        
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, predictions=masked_lm_example_loss, scaffold_fn=scaffold_fn) 
    else:
      raise ValueError("Only TRAIN and EVAL modes are supported: %s" % (mode))

    return output_spec

  return model_fn

def get_masked_lm_output(bert_config, input_tensor, output_weights, positions,
                         label_ids, label_weights):
  """Get loss and log probs for the masked LM."""
  input_tensor = gather_indexes(input_tensor, positions)

  with tf.variable_scope("cls/predictions"):
    # We apply one more non-linear transformation before the output layer.
    # This matrix is not used after pre-training.
    with tf.variable_scope("transform"):
      input_tensor = tf.layers.dense(
          input_tensor,
          units=bert_config.hidden_size,
          activation=model_1.get_activation(bert_config.hidden_act),
          kernel_initializer=model_1.create_initializer(
              bert_config.initializer_range))
      input_tensor = model_1.layer_norm(input_tensor)

    # The output weights are the same as the input embeddings, but there is
    # an output-only bias for each token.
    output_bias = tf.get_variable(
        "output_bias",
        shape=[bert_config.vocab_size],
        initializer=tf.zeros_initializer())
    logits = tf.matmul(input_tensor, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)
    label_ids = tf.reshape(label_ids, [-1])
    label_weights = tf.reshape(label_weights, [-1])

    one_hot_labels = tf.one_hot(
        label_ids, depth=bert_config.vocab_size, dtype=tf.float32)

    # The `positions` tensor might be zero-padded (if the sequence is too
    # short to have the maximum number of predictions). The `label_weights`
    # tensor has a value of 1.0 for every real prediction and 0.0 for the
    # padding predictions.
    per_example_loss = -tf.reduce_sum(log_probs * one_hot_labels, axis=[-1])
    #numerator = tf.reduce_sum(label_weights * per_example_loss)
    #denominator = tf.reduce_sum(label_weights) + 1e-5
    #loss = numerator / denominator
    loss = tf.reshape(per_example_loss, [-1, tf.shape(positions)[1]])
    #print(loss.shape)

  return loss

def gather_indexes(sequence_tensor, positions):
  """Gathers the vectors at the specific positions over a minibatch."""
  sequence_shape = model_1.get_shape_list(sequence_tensor, expected_rank=3)
  batch_size = sequence_shape[0]
  seq_length = sequence_shape[1]
  width = sequence_shape[2]

  flat_offsets = tf.reshape(
      tf.range(0, batch_size, dtype=tf.int32) * seq_length, [-1, 1])
  flat_positions = tf.reshape(positions + flat_offsets, [-1])
  flat_sequence_tensor = tf.reshape(sequence_tensor,
                                    [batch_size * seq_length, width])
  output_tensor = tf.gather(flat_sequence_tensor, flat_positions)
  return output_tensor

def input_fn_builder(input_files,
                     max_seq_length,
                     max_predictions_per_seq,
                     is_training,
                     num_cpu_threads=4,
                     hvd=None):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    name_to_features = {
        "input_ids":
            tf.FixedLenFeature([max_seq_length], tf.int64),
        "input_mask":
            tf.FixedLenFeature([max_seq_length], tf.int64),
        #"segment_ids":
         #   tf.FixedLenFeature([max_seq_length], tf.int64),
        "masked_lm_positions":
            tf.FixedLenFeature([max_predictions_per_seq], tf.int64),
        "masked_lm_ids":
            tf.FixedLenFeature([max_predictions_per_seq], tf.int64),
        "masked_lm_weights":
            tf.FixedLenFeature([max_predictions_per_seq], tf.float32),
    }

    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    if is_training:
      d = tf.data.Dataset.from_tensor_slices(tf.constant(input_files))
      if hvd is not None: d = d.shard(hvd.size(), hvd.rank())
      d = d.repeat()
      d = d.shuffle(buffer_size=len(input_files))

      # `cycle_length` is the number of parallel files that get read.
      cycle_length = min(num_cpu_threads, len(input_files))

      # `sloppy` mode means that the interleaving is not exact. This adds
      # even more randomness to the training pipeline.
      d = d.apply(
          tf.contrib.data.parallel_interleave(
              tf.data.TFRecordDataset,
              sloppy=is_training,
              cycle_length=cycle_length))
      d = d.shuffle(buffer_size=100)
    else:
      d = tf.data.TFRecordDataset(input_files)
      # Since we evaluate for a fixed number of steps we don't want to encounter
      # out-of-range exceptions.
      d = d.repeat()

    # We must `drop_remainder` on training because the TPU requires fixed
    # size dimensions. For eval, we assume we are evaluating on the CPU or GPU
    # and we *don't* want to drop the remainder, otherwise we wont cover
    # every sample.
    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            num_parallel_batches=num_cpu_threads,
            drop_remainder=True))#Default was True
    return d

  return input_fn

def _decode_record(record, name_to_features):
  """Decodes a record to a TensorFlow example."""
  example = tf.parse_single_example(record, name_to_features)

   #tf.Example only supports tf.int64, but the TPU only supports tf.int32.
  # So cast all int64 to int32.
  for name in list(example.keys()):
    t = example[name]
    if t.dtype == tf.int64:
      t = tf.to_int32(t)
    example[name] = t

  return example



# This function is not used by this file but is still used by the Colab and
# people who depend on it.
def convert_examples_to_features(examples, max_seq_length, tokenizer):
  """Convert a set of `InputExample`s to a list of `InputFeatures`."""

  all_features = []
  all_tokens = []

  for (ex_index, example) in enumerate(examples):
    if ex_index % 10000 == 0:
      tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    features, tokens = convert_single_example(ex_index, example,
                                     max_seq_length, tokenizer)
    all_features.extend(features)
    all_tokens.extend(tokens)

  return all_features, all_tokens



def create_masked_lm_prediction(input_ids, mask_position, mask_count=1):
  new_input_ids = list(input_ids)
  masked_lm_labels = []
  masked_lm_positions = list(range(mask_position, mask_position + mask_count))
  for i in masked_lm_positions:
    new_input_ids[i] = MASKED_ID
    masked_lm_labels.append(input_ids[i])
  return new_input_ids, masked_lm_positions, masked_lm_labels


class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self, input_ids, segment_ids, input_mask, masked_lm_positions,
               masked_lm_ids):
    self.input_ids = input_ids,
    self.segment_ids = segment_ids,
    self.input_mask = input_mask,
    self.masked_lm_positions = masked_lm_positions,
    self.masked_lm_ids = masked_lm_ids,


def convert_single_example(ex_index, example, max_seq_length,
                           tokenizer):
  """Converts a single `InputExample` into a single `InputFeatures`."""
  tokens = tokenizer.tokenize(example.text)

  # Account for [CLS] and [SEP] with "- 2"
  if len(tokens) > max_seq_length - 2:
    tokens = tokens[0:(max_seq_length - 2)]

  input_tokens = []
  segment_ids = []
  input_tokens.append("[CLS]")
  segment_ids.append(0)
  for token in tokens:
    input_tokens.append(token)
    segment_ids.append(0)
  input_tokens.append("[SEP]")
  segment_ids.append(0)

  input_ids = tokenizer.convert_tokens_to_ids(input_tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  if ex_index < 5:
    tf.logging.info("*** Example ***")
    tf.logging.info("id: %s" % (example.unique_id))
    tf.logging.info("tokens: %s" % " ".join(
        [tokenization.printable_text(x) for x in input_tokens]))
    tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
    tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
    tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))

  features = create_sequential_mask(input_tokens, input_ids, input_mask, segment_ids,
                                    FLAGS.max_predictions_per_seq)
  return features, input_tokens


def create_sequential_mask(input_tokens, input_ids, input_mask, segment_ids,
                           max_predictions_per_seq):
  """Mask each token/word sequentially"""
  features = []
  i = 1
  while i < len(input_tokens) - 1:
    mask_count = 1
    while is_subtoken(input_tokens[i+mask_count]):
      mask_count += 1

    input_ids_new, masked_lm_positions, masked_lm_labels = create_masked_lm_prediction(input_ids, i, mask_count)
    while len(masked_lm_positions) < max_predictions_per_seq:
      masked_lm_positions.append(0)
      masked_lm_labels.append(0)

    feature = InputFeatures(
      input_ids=input_ids_new,
      input_mask=input_mask,
      segment_ids=segment_ids,
      masked_lm_positions=masked_lm_positions,
      masked_lm_ids=masked_lm_labels)
    features.append(feature)
    i += mask_count
  return features

def is_subtoken(x):
  return '+' in x

def parse_result(result, all_tokens, output_file=None):
  with tf.gfile.GFile(output_file, "w") as writer:
    tf.logging.info("***** Predict results *****")
    i = 0
    sentences = []
    counter =0
    for word_loss in result:
        counter+=1
    print(counter)
    for word_loss in result:
      #print(word_loss)
      # start of a sentence
      if all_tokens[i] == "[CLS]":
        sentence = {}
        tokens = []
        sentence_loss = 0.0
        word_count_per_sent = 0
        i += 1

      # add token
      tokens.append({"token": printable_text(all_tokens[i]),
                     "prob": float(np.exp(-word_loss[0])) })
      sentence_loss += word_loss[0]
      word_count_per_sent += 1
      i += 1

      token_count_per_word = 0
      while is_subtoken(all_tokens[i]):
        token_count_per_word += 1
        tokens.append({"token": printable_text(all_tokens[i]),
                       "prob": float(np.exp(-word_loss[token_count_per_word]))})
        sentence_loss += word_loss[token_count_per_word]
        i += 1

      # end of a sentence
      if all_tokens[i] == "[SEP]":
        sentence["tokens"] = tokens
        sentence["ppl"] = float(np.exp(sentence_loss / word_count_per_sent))
        sentences.append(sentence)
        i += 1
        break

    if output_file is not None:
      tf.logging.info("Saving results to %s" % output_file)
      writer.write(json.dumps(sentences, indent=2, ensure_ascii=False))


def main(_):
    tf.logging.set_verbosity(tf.logging.INFO)
    bert_config = model_1.BertConfig.from_json_file('bert_config_1506.json')
    input_files = []
    input_files.extend(tf.gfile.Glob('yle_examplesentence.tfrecord'))
    tf.logging.info("*** Input Files ***")
    for input_file in input_files:
        tf.logging.info("  %s" % input_file)
    
    
    config = tf.ConfigProto()
    '''
    The TensorFlow/XLA JIT compiler compiles and runs parts of TensorFlow graphs via XLA. The benefit of this over the standard TensorFlow implementation is that XLA can fuse multiple operators (kernel fusion) into a small number of compiled kernels. Fusing operators can reduce memory bandwidth requirements and improve performance compared to executing operators one-at-a-time, as the TensorFlow executor does.'''
    tpu_cluster_resolver = None
    #without XLA
    #config.graph_options.optimizer_options.global_jit_level = tf.OptimizerOptions.ON_1
    is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
    run_config = tf.contrib.tpu.RunConfig(
        cluster=tpu_cluster_resolver,
        master=None,
        model_dir='outputckpoints_1506',
        session_config=config,
        save_checkpoints_steps=5000,
        tpu_config=tf.contrib.tpu.TPUConfig(
            iterations_per_loop=1000,
            num_shards=8,
            per_host_input_for_training=is_per_host),
        # This variable controls how often estimator reports examples/sec.
        # Default value is every 100 steps.
        # When --report_loss is True, we set to very large value to prevent
        # default info reporting from estimator.
        # Ideally we should set it to None, but that does not work.
        log_step_count_steps=10000)
    model_fn = model_fn_builder(
        bert_config=bert_config,
        init_checkpoint=None,
        learning_rate=1e-4,
        num_train_steps=300000,
        num_warmup_steps=10000,
        use_tpu=False,
        use_one_hot_embeddings=False,
        hvd=None)
    training_hooks = []
    training_hooks.append(_LogSessionRunHook(32,5000,-1))
    estimator = tf.contrib.tpu.TPUEstimator(
        use_tpu=False,
        model_fn=model_fn,
        config=run_config,
        train_batch_size=32,
        eval_batch_size=32,
        predict_batch_size=32)
    if FLAGS.do_train:
        tf.logging.info("***** Running training *****")
        tf.logging.info("  Batch size = %d",32)
        train_input_fn = input_fn_builder(
            input_files=input_files,
            max_seq_length=300,
            max_predictions_per_seq=40,
            is_training=True,
            hvd=None )
        estimator.train(input_fn=train_input_fn, hooks=training_hooks, max_steps=300000)
    
    tf.logging.info("***** Running evaluation *****")
    tf.logging.info("  Batch size = %d", 2)

    eval_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=300,
        max_predictions_per_seq=40,
        is_training=False)
    result = estimator.predict(input_fn=eval_input_fn)
    output_predict_file = os.path.join('/result_2806', "test_results.json")
    for res in result:
        print ('result: {}'.format(result))
    #print(result.shape)
    #parse_result(result, tokk, output_predict_file)

    #result = estimator.evaluate(
     #   input_fn=eval_input_fn, steps=1)

    #output_eval_file = os.path.join('outputckpoints_1506', "eval_results_yleexampledev.txt")
    #with tf.gfile.GFile(output_predict_file, "w") as writer:
      #tf.logging.info("***** Eval results *****")
      #for key in sorted(result.keys()):
        #tf.logging.info("  %s = %s", key, str(result[key]))
        #writer.write("%s = %s\n" % (key, str(result[key])))
        
    

if __name__ == "__main__":
    tf.app.run()


INFO:tensorflow:*** Input Files ***
INFO:tensorflow:  ./yle_examplesentence.tfrecord
INFO:tensorflow:Using config: {'_model_dir': 'outputckpoints_1506', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': , '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f40c02454a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_di

INFO:tensorflow:  0 :: name = bert/encoder/layer_3/output/dense/bias:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_3/output/LayerNorm/beta:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_3/output/LayerNorm/gamma:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_4/attention/self/query/kernel:0, shape = (896, 896)
INFO:tensorflow:  0 :: name = bert/encoder/layer_4/attention/self/query/bias:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_4/attention/self/key/kernel:0, shape = (896, 896)
INFO:tensorflow:  0 :: name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (896, 896)
INFO:tensorflow:  0 :: name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (896, 896)
INFO:tensorflow:  0 :: name = bert/encoder/

INFO:tensorflow:  0 :: name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_9/attention/self/value/kernel:0, shape = (896, 896)
INFO:tensorflow:  0 :: name = bert/encoder/layer_9/attention/self/value/bias:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_9/attention/output/dense/kernel:0, shape = (896, 896)
INFO:tensorflow:  0 :: name = bert/encoder/layer_9/attention/output/dense/bias:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_9/attention/output/LayerNorm/beta:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_9/attention/output/LayerNorm/gamma:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_9/intermediate/dense/kernel:0, shape = (896, 3584)
INFO:tensorflow:  0 :: name = bert/encoder/layer_9/intermediate/dense/bias:0, shape = (3584,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_9/output/dense/kernel:0, shape = (3584, 896)
INFO:tensorflow:  0 

INFO:tensorflow:  0 :: name = bert/encoder/layer_14/attention/output/LayerNorm/beta:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_14/attention/output/LayerNorm/gamma:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_14/intermediate/dense/kernel:0, shape = (896, 3584)
INFO:tensorflow:  0 :: name = bert/encoder/layer_14/intermediate/dense/bias:0, shape = (3584,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_14/output/dense/kernel:0, shape = (3584, 896)
INFO:tensorflow:  0 :: name = bert/encoder/layer_14/output/dense/bias:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_14/output/LayerNorm/beta:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_14/output/LayerNorm/gamma:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_15/attention/self/query/kernel:0, shape = (896, 896)
INFO:tensorflow:  0 :: name = bert/encoder/layer_15/attention/self/query/bias:0, shape = (896,)
INFO:tensorflow:  0 :: name = ber

INFO:tensorflow:  0 :: name = bert/encoder/layer_19/output/dense/kernel:0, shape = (3584, 896)
INFO:tensorflow:  0 :: name = bert/encoder/layer_19/output/dense/bias:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_19/output/LayerNorm/beta:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/encoder/layer_19/output/LayerNorm/gamma:0, shape = (896,)
INFO:tensorflow:  0 :: name = bert/pooler/dense/kernel:0, shape = (896, 896)
INFO:tensorflow:  0 :: name = bert/pooler/dense/bias:0, shape = (896,)
INFO:tensorflow:  0 :: name = cls/predictions/transform/dense/kernel:0, shape = (896, 896)
INFO:tensorflow:  0 :: name = cls/predictions/transform/dense/bias:0, shape = (896,)
INFO:tensorflow:  0 :: name = cls/predictions/transform/LayerNorm/beta:0, shape = (896,)
INFO:tensorflow:  0 :: name = cls/predictions/transform/LayerNorm/gamma:0, shape = (896,)
INFO:tensorflow:  0 :: name = cls/predictions/output_bias:0, shape = (34522,)
INFO:tensorflow:Done calling model_fn.
INFO:tensorf

...

result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <g

result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <g

result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <g

result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <generator object TPUEstimator.predict at 0x7f40f96e2b88>
result: <g

KeyboardInterrupt: 

In [70]:
log_probas

[]

In [118]:
is_subtoken(' ss')

False